### k-means

In [1]:
import numpy as np

In [2]:
def init_centroid(X, k, n_data):
    # 各データ点の中からセントロイドとなる点をk個ランダムに選択
    idx = np.random.permutation(n_data)[:k]
    centroids = X[idx]
    return centroids

In [3]:
def compute_distances(X, k, n_data, centroids):
    distances = np.zeros((n_data, k))
    for idx_centroids in range(k):
        dist = np.sqrt(np.sum((X - centroids[idx_centroids]) ** 2, axis=1))
        distances[:, idx_centroids] = dist
    return distances

In [19]:
def k_means(k, X, max_iter=300):
    '''
    X.shape = (データ数、次元数)
    k = クラスタ数
    '''
    n_data, n_features = X.shape

    # セントロイドの初期値
    centroids = init_centroid(X, n_data, k)

    # 新しいクラスタを格納するための配列
    new_cluster = np.zeros(n_data)

    # 各データの所属クラスタを保存する配列
    cluster = np.zeros(n_data)

    for epoch in range(max_iter):
        # 各データ点とセントロイドとの距離を計算
        distances = compute_distances(X, k, n_data, centroids)

        # 新たな所属クラスタを計算
        new_cluster = np.argmin(distances, axis=1)
        # print('new_cluster', new_cluster)

        # 全てのクラスタに対してセントロイドを再計算
        for idx_centroids in range(k):
            centroids[idx_centroids] = X[new_cluster == idx_centroids].mean(axis=0)
            # print('セントロイド', X[new_cluster == idx_centroids])
            # print('セントロイド', X[new_cluster == idx_centroids].mean(axis=0))
        
        # クラスタによるグループ分けに変化がなかったら終了
        if (new_cluster == cluster).all():
            break
        
        cluster = new_cluster
    
    return cluster

In [20]:
# seed値固定
np.random.seed(874)
# x座標
x = np.r_[np.random.normal(size=1000,loc=0,scale=1),
          np.random.normal(size=1000,loc=4,scale=1)]
# y座標
y = np.r_[np.random.normal(size=1000,loc=10,scale=1),
          np.random.normal(size=1000,loc=10,scale=1)]
data = np.c_[x, y]
print(data.shape)

(2000, 2)


In [21]:
print(k_means(2, data))

new_cluster [0 1 1 ... 1 1 1]
セントロイド [[-0.20326761 11.11519782]
 [-1.4092337  10.62356966]
 [-0.65379406 10.11412878]
 ...
 [ 1.06946839 10.36416877]
 [ 5.08669304 11.77011704]
 [ 1.87238621 10.62203165]]
セントロイド [ 0.45547013 10.90751717]
セントロイド [[ 0.51059539  8.9471941 ]
 [-0.081612    9.13104393]
 [ 0.47263178  9.8013869 ]
 ...
 [ 4.88229755  9.12563974]
 [ 5.15585889  9.74081804]
 [ 3.69871148 10.7152379 ]]
セントロイド [2.67407055 9.61807376]
new_cluster [0 0 0 ... 1 1 1]
セントロイド [[-0.20326761 11.11519782]
 [ 0.51059539  8.9471941 ]
 [-0.081612    9.13104393]
 ...
 [ 1.67659235 10.55529045]
 [ 2.31646533 11.61950896]
 [ 1.06946839 10.36416877]]
セントロイド [-0.16180067 10.1117885 ]
セントロイド [[ 1.58816583  8.69074879]
 [ 2.44456484 11.4088265 ]
 [ 0.23223802  7.37433972]
 ...
 [ 4.88229755  9.12563974]
 [ 5.15585889  9.74081804]
 [ 3.69871148 10.7152379 ]]
セントロイド [3.79330613 9.94313832]
new_cluster [0 0 0 ... 1 1 1]
セントロイド [[-0.20326761 11.11519782]
 [ 0.51059539  8.9471941 ]
 [-0.081612    9.1310

### k-means++

In [ ]:
def init_centroid(X, k, n_data):
    # 1つ目のセントロイドをランダムに選択
    idx = np.random.choice(n_data, 1)
    centroids = X[idx]
    for i in range(k - 1):
        # 各データ点とセントロイドとの距離を計算
        distances = compute_distances(X, len(centroids), n_data, centroids)

        # 各データ点と最も近いセントロイドとの距離の二乗を計算
        closet_dist_sq = np.min(distances ** 2, axis=1)

        # 距離の二乗の和を計算
        weights = closet_dist_sq.sum()

        # [0, 1)の乱数と距離の二乗和を掛ける
        rand_vals = np.random.random_sample() * weights

        # 距離の二乗の累積和を計算し、rand_valと最も値が近いデータ点のindexを取得
        candidate_idx = np.searchsorted(np.cumsum(closet_dist_sq), rand_vals)

        # 選ばれた点を新たなセントロイドとして追加
        centroids = np.vstack([centroids, X[candidate_idx]])

        return centroids

### 主成分分析

In [ ]:
def pca(X, n_components=2):
    # データから平均を引く
    X = X - X.mean(axis=0)

    #  共分散行列の作成
    cov = np.cov(X, rowvar=False)

    # 固有値や主成分方向を計算
    l, v = np.linalg.eig(cov)

    # 固有値の大きい順に並び替え
    l_index = np.argsort(l)[::-1]
    v_ = v[:, l_index]

    # n_components分、主成分方向を取得
    components = v_[:, :n_components]

    # データを低次元空間へ射影
    T = np.dot(X, components)

    return T